# Project: Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, you'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

![](_post-hmm.png)

The notebook already contains some code to get you started. You only need to add some new functionality in the areas indicated to complete the project; you will not need to modify the included code beyond what is requested. Sections that begin with **'IMPLEMENTATION'** in the header indicate that you must provide code in the block that follows. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

<div class="alert alert-block alert-info">
**Note:** Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You must then **export the notebook** by running the last cell in the notebook, or by using the menu above and navigating to **File -> Download as -> HTML (.html)** Your submissions should include both the `html` and `ipynb` files.
</div>

<div class="alert alert-block alert-info">
**Note:** Code and Markdown cells can be executed using the `Shift + Enter` keyboard shortcut. Markdown cells can be edited by double-clicking the cell to enter edit mode.
</div>

### The Road Ahead
You must complete Steps 1-3 below to pass the project. The section on Step 4 includes references & resources you can use to further explore HMM taggers.

- [Step 1](#Step-1:-Read-and-preprocess-the-dataset): Review the provided interface to load and access the text corpus
- [Step 2](#Step-2:-Build-a-Most-Frequent-Class-tagger): Build a Most Frequent Class tagger to use as a baseline
- [Step 3](#Step-3:-Build-an-HMM-tagger): Build an HMM Part of Speech tagger and compare to the MFC baseline
- [Step 4](#Step-4:-[Optional]-Improving-model-performance): (Optional) Improve the HMM tagger

In [1]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

In [2]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

## Step 1: Read and preprocess the dataset
---
We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), but the process you'll follow would be the same.

The `Dataset` class provided in helpers.py will read and parse the corpus. You can generate your own datasets compatible with the reader by writing them to the following format. The dataset is stored in plaintext as a collection of words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Example from the Brown corpus. 
```
b100-38532
Perhaps	ADV
it	PRON
was	VERB
right	ADJ
;	.
;	.

b100-35577
...
```

In [3]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


### The Dataset Interface

You can access (mostly) immutable references to the dataset through a simple interface provided through the `Dataset` class, which represents an iterable collection of sentences along with easy access to partitions of the data for training & testing. Review the reference below, then run and review the next few cells to make sure you understand the interface before moving on to the next step.

```
Dataset-only Attributes:
    training_set - reference to a Subset object containing the samples for training
    testing_set - reference to a Subset object containing the samples for testing

Dataset & Subset Attributes:
    sentences - a dictionary with an entry {sentence_key: Sentence()} for each sentence in the corpus
    keys - an immutable ordered (not sorted) collection of the sentence_keys for the corpus
    vocab - an immutable collection of the unique words in the corpus
    tagset - an immutable collection of the unique tags in the corpus
    X - returns an array of words grouped by sentences ((w11, w12, w13, ...), (w21, w22, w23, ...), ...)
    Y - returns an array of tags grouped by sentences ((t11, t12, t13, ...), (t21, t22, t23, ...), ...)
    N - returns the number of distinct samples (individual words or tags) in the dataset

Methods:
    stream() - returns an flat iterable over all (word, tag) pairs across all sentences in the corpus
    __iter__() - returns an iterable over the data as (sentence_key, Sentence()) pairs
    __len__() - returns the nubmer of sentences in the dataset
```

For example, consider a Subset, `subset`, of the sentences `{"s0": Sentence(("See", "Spot", "run"), ("VERB", "NOUN", "VERB")), "s1": Sentence(("Spot", "ran"), ("NOUN", "VERB"))}`. The subset will have these attributes:

```
subset.keys == {"s1", "s0"}  # unordered
subset.vocab == {"See", "run", "ran", "Spot"}  # unordered
subset.tagset == {"VERB", "NOUN"}  # unordered
subset.X == (("Spot", "ran"), ("See", "Spot", "run"))  # order matches .keys
subset.Y == (("NOUN", "VERB"), ("VERB", "NOUN", "VERB"))  # order matches .keys
subset.N == 7  # there are a total of seven observations over all sentences
len(subset) == 2  # because there are two sentences
```

<div class="alert alert-block alert-info">
**Note:** The `Dataset` class is _convenient_, but it is **not** efficient. It is not suitable for huge datasets because it stores multiple redundant copies of the same data.
</div>

#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [4]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


<div class="alert alert-block alert-info">
**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.
</div>

#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [5]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [6]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [7]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')



For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts. 

## Step 2: Build a Most Frequent Class tagger
---

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

### IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

In [8]:
# Extra - not necessary for submission
for tag, key in zip([1],[2]):
    print(tag,key)

1 2


In [9]:
len(data.training_set.Y)

45872

In [185]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    # TODO: Finish this function!
    solution_dict = defaultdict(int)
    
    for i in range(len(sequences_A)):
        for tag_i, word_i in zip(sequences_A[i], sequences_B[i]):
            if tag_i not in solution_dict:
                solution_dict[tag_i] = {} # create a nested dictionary
                solution_dict[tag_i][word_i] = solution_dict[tag_i].get(word_i, 0) + 1
            else: # so tag is now in the solution_dict!
                solution_dict[tag_i][word_i] = solution_dict[tag_i].get(word_i, 0) + 1
    
#     print(solution_dict) # only run during testing

    return solution_dict

    raise NotImplementedError


# Calculate C(t_i, w_i)
# emission_counts = pair_counts(data.training_set.Y, data.training_set.X)
emission_counts = pair_counts(data.Y, data.X)
# TODO: YOUR CODE HERE)

assert len(emission_counts) == 12, \
       "Uh oh. There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

### IMPLEMENTATION: Most Frequent Class Tagger

Use the `pair_counts()` function and the training dataset to find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [11]:
defaultdict()

defaultdict(None, {})

In [12]:
len(data.training_set.vocab)

50536

In [186]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

# TODO: YOUR CODE HERE)

word_counts = pair_counts(data.training_set.X, data.training_set.Y)

# TODO: YOUR CODE HERE
mfc_table = defaultdict() 

for word, tags in word_counts.items():
    freq_tag = max(tags.keys(), key=lambda key: tags[key])
    mfc_table[word] = freq_tag 

# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

assert len(mfc_table) == len(data.training_set.vocab), ""
assert all(k in data.training_set.vocab for k in mfc_table.keys()), ""
assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, ""
HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')

### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

In [187]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [188]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [189]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [190]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.01%


## Step 3: Build an HMM tagger
---
The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where you'll implement functions to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t_i^n = \underset{t_i^n}{\mathrm{argmax}} \prod_{i=1}^n P(w_i|t_i) P(t_i|t_{i-1})$$

Refer to Speech & Language Processing [Chapter 10](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

### IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [25]:
data.training_set.stream()

In [201]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    # TODO: Finish this function!
    unigram_results = defaultdict(int)
    
    for seq in sequences:
        # print(seq) # only for debugging
        for specific_tag in seq:
            unigram_results[specific_tag] += 1
    
    return unigram_results
    
    raise NotImplementedError

# TODO: call unigram_counts with a list of tag sequences from the training set
tag_unigrams = unigram_counts(data.Y)# TODO: YOUR CODE HERE)

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')

### IMPLEMENTATION: Bigram Counts

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [192]:
for i, pair in enumerate(data.training_set.stream()):
    print(i,pair)

0 ('Whenever', 'ADV')
1 ('artists', 'NOUN')
2 (',', '.')
3 ('indeed', 'ADV')
4 (',', '.')
5 ('turned', 'VERB')
6 ('to', 'ADP')
7 ('actual', 'ADJ')
8 ('representations', 'NOUN')
9 ('or', 'CONJ')
10 ('molded', 'VERB')
11 ('three-dimensional', 'ADJ')
12 ('figures', 'NOUN')
13 (',', '.')
14 ('which', 'DET')
15 ('were', 'VERB')
16 ('rare', 'ADJ')
17 ('down', 'PRT')
18 ('to', 'ADP')
19 ('800', 'NUM')
20 ('B.C.', 'NOUN')
21 (',', '.')
22 ('they', 'PRON')
23 ('tended', 'VERB')
24 ('to', 'PRT')
25 ('reflect', 'VERB')
26 ('reality', 'NOUN')
27 ('(', '.')
28 ('see', 'VERB')
29 ('Plate', 'NOUN')
30 ('6a', 'NUM')
31 (',', '.')
32 ('9b', 'NUM')
33 (')', '.')
34 (';', '.')
35 (';', '.')
36 ('For', 'ADP')
37 ('almost', 'ADV')
38 ('two', 'NUM')
39 ('months', 'NOUN')
40 (',', '.')
41 ('the', 'DET')
42 ('defendant', 'NOUN')
43 ('and', 'CONJ')
44 ('the', 'DET')
45 ('world', 'NOUN')
46 ('heard', 'VERB')
47 ('from', 'ADP')
48 ('individuals', 'NOUN')
49 ('escaped', 'VERB')
50 ('from', 'ADP')
51 ('the', 'DET'

892 ('telephoned', 'VERB')
893 ('Torrio', 'NOUN')
894 ('and', 'CONJ')
895 ('returned', 'VERB')
896 ('to', 'ADP')
897 ("O'Banion", 'NOUN')
898 ('with', 'ADP')
899 ('the', 'DET')
900 ('message', 'NOUN')
901 (':', '.')
902 ('``', '.')
903 ('Say', 'PRT')
904 (',', '.')
905 ('Dionie', 'NOUN')
906 (',', '.')
907 ('I', 'PRON')
908 ('just', 'ADV')
909 ('been', 'VERB')
910 ('talking', 'VERB')
911 ('to', 'ADP')
912 ('Johnny', 'NOUN')
913 (',', '.')
914 ('and', 'CONJ')
915 ('he', 'PRON')
916 ('said', 'VERB')
917 ('to', 'PRT')
918 ('let', 'VERB')
919 ('them', 'PRON')
920 ('cops', 'NOUN')
921 ('have', 'VERB')
922 ('the', 'DET')
923 ('three', 'NUM')
924 ('hundred', 'NUM')
925 ('.', '.')
926 ('``', '.')
927 ('I', 'PRON')
928 ('hit', 'VERB')
929 ('him', 'PRON')
930 ('in', 'ADP')
931 ('the', 'DET')
932 ('ass', 'NOUN')
933 ('once', 'ADV')
934 ("''", '.')
935 ('.', '.')
936 ('but', 'CONJ')
937 ('this', 'DET')
938 ('obligation', 'NOUN')
939 ('pertains', 'VERB')
940 ('only', 'ADV')
941 ('to', 'ADP')
942 ('

1642 ('public', 'NOUN')
1643 ('.', '.')
1644 ('My', 'DET')
1645 ("name's", 'PRT')
1646 ('Carla', 'NOUN')
1647 ('Caneli', 'NOUN')
1648 ('.', '.')
1649 ('Beyond', 'ADP')
1650 ('the', 'DET')
1651 ('stockade', 'NOUN')
1652 ('rifles', 'NOUN')
1653 ('began', 'VERB')
1654 ('to', 'PRT')
1655 ('explode', 'VERB')
1656 ('as', 'ADP')
1657 ('some', 'DET')
1658 ('of', 'ADP')
1659 ('the', 'DET')
1660 ('guerrillas', 'NOUN')
1661 ('fired', 'VERB')
1662 ('at', 'ADP')
1663 ('shadows', 'NOUN')
1664 ('that', 'ADP')
1665 ('they', 'PRON')
1666 ('imagined', 'VERB')
1667 ('were', 'VERB')
1668 ('Apaches', 'NOUN')
1669 ('.', '.')
1670 ('The', 'DET')
1671 ('deadlock', 'NOUN')
1672 ('has', 'VERB')
1673 ('been', 'VERB')
1674 ('caused', 'VERB')
1675 ('by', 'ADP')
1676 ('the', 'DET')
1677 ("Russians'", 'NOUN')
1678 ('new', 'ADJ')
1679 ('demand', 'NOUN')
1680 ('for', 'ADP')
1681 ('a', 'DET')
1682 ('three-man', 'ADJ')
1683 ('(', '.')
1684 ('East', 'NOUN')
1685 (',', '.')
1686 ('West', 'ADJ')
1687 ('and', 'CONJ')
1688 (

2391 (',', '.')
2392 ('so', 'ADV')
2393 ('useful', 'ADJ')
2394 (',', '.')
2395 ('in', 'ADP')
2396 ('fact', 'NOUN')
2397 (',', '.')
2398 ('that', 'ADP')
2399 ('they', 'PRON')
2400 ('have', 'VERB')
2401 ('been', 'VERB')
2402 ('successfully', 'ADV')
2403 ('used', 'VERB')
2404 ('to', 'PRT')
2405 ('suggest', 'VERB')
2406 ('ways', 'NOUN')
2407 ('to', 'PRT')
2408 ('speed', 'VERB')
2409 ('diagnosis', 'NOUN')
2410 ('and', 'CONJ')
2411 ('to', 'PRT')
2412 ('evaluate', 'VERB')
2413 ('the', 'DET')
2414 ('progress', 'NOUN')
2415 ('of', 'ADP')
2416 ('therapy', 'NOUN')
2417 ('.', '.')
2418 ('The', 'DET')
2419 ('bill', 'NOUN')
2420 ('which', 'DET')
2421 ('passed', 'VERB')
2422 ('the', 'DET')
2423 ('Assembly', 'NOUN')
2424 ('last', 'ADJ')
2425 ('May', 'NOUN')
2426 ('and', 'CONJ')
2427 ('is', 'VERB')
2428 ('now', 'ADV')
2429 ('pending', 'ADJ')
2430 ('in', 'ADP')
2431 ('the', 'DET')
2432 ('Senate', 'NOUN')
2433 ('should', 'VERB')
2434 ('be', 'VERB')
2435 ('given', 'VERB')
2436 ('careful', 'ADJ')
2437 ('sc

3141 ('M.', 'NOUN')
3142 ("Forbes's", 'NOUN')
3143 ('Paget', 'NOUN')
3144 (',', '.')
3145 ('which', 'DET')
3146 ('had', 'VERB')
3147 ('what', 'DET')
3148 ('seemed', 'VERB')
3149 ('to', 'PRT')
3150 ('be', 'VERB')
3151 ('a', 'DET')
3152 ('substantial', 'ADJ')
3153 ('lead', 'NOUN')
3154 ('in', 'ADP')
3155 ('the', 'DET')
3156 ('early', 'ADJ')
3157 ('stages', 'NOUN')
3158 (',', '.')
3159 ('tired', 'VERB')
3160 ('rapidly', 'ADV')
3161 ('nearing', 'VERB')
3162 ('the', 'DET')
3163 ('wire', 'NOUN')
3164 ('and', 'CONJ')
3165 ('was', 'VERB')
3166 ('able', 'ADJ')
3167 ('to', 'PRT')
3168 ('save', 'VERB')
3169 ('place', 'NOUN')
3170 ('money', 'NOUN')
3171 ('only', 'ADV')
3172 ('a', 'DET')
3173 ('head', 'NOUN')
3174 ('in', 'ADP')
3175 ('front', 'NOUN')
3176 ('of', 'ADP')
3177 ('Glen', 'NOUN')
3178 ('T.', 'NOUN')
3179 ("Hallowell's", 'NOUN')
3180 ('Milties', 'NOUN')
3181 ('Miss', 'NOUN')
3182 ('.', '.')
3183 ('He', 'PRON')
3184 ('guessed', 'VERB')
3185 ('by', 'ADP')
3186 ('their', 'DET')
3187 ('appear

...

84860 ('answers', 'NOUN')
84861 ('on', 'ADP')
84862 ('when', 'ADV')
84863 ('to', 'PRT')
84864 ('do', 'VERB')
84865 ('what', 'DET')
84866 ('at', 'ADP')
84867 ('the', 'DET')
84868 ('site', 'NOUN')
84869 ('.', '.')
84870 ('In', 'ADP')
84871 ('addition', 'NOUN')
84872 ('to', 'ADP')
84873 ('the', 'DET')
84874 ('regular', 'ADJ')
84875 ('schedule', 'NOUN')
84876 (',', '.')
84877 ('advertisements', 'NOUN')
84878 ('were', 'VERB')
84879 ('run', 'VERB')
84880 ('for', 'ADP')
84881 ('maximum', 'ADJ')
84882 ('impact', 'NOUN')
84883 ('in', 'ADP')
84884 ('special', 'ADJ')
84885 ('editions', 'NOUN')
84886 ('of', 'ADP')
84887 ('the', 'DET')
84888 ('New', 'ADJ')
84889 ('York', 'NOUN')
84890 ('Times', 'NOUN')
84891 (',', '.')
84892 ('Boston', 'NOUN')
84893 ('Herald', 'NOUN')
84894 (',', '.')
84895 ('American', 'ADJ')
84896 ('Banker', 'NOUN')
84897 (',', '.')
84898 ('Electronic', 'ADJ')
84899 ('News', 'NOUN')
84900 ('and', 'CONJ')
84901 (',', '.')
84902 ('for', 'ADP')
84903 ('local', 'ADJ')
84904 ('promoti

85653 ('method', 'NOUN')
85654 (',', '.')
85655 ('their', 'DET')
85656 ('operations', 'NOUN')
85657 ('would', 'VERB')
85658 ('begin', 'VERB')
85659 ('to', 'PRT')
85660 ('affect', 'VERB')
85661 ('stock', 'NOUN')
85662 ('prices', 'NOUN')
85663 (',', '.')
85664 ('and', 'CONJ')
85665 ('thus', 'ADV')
85666 ('throw', 'VERB')
85667 ('the', 'DET')
85668 ('charts', 'NOUN')
85669 ('off', 'PRT')
85670 ('.', '.')
85671 ('Hold', 'VERB')
85672 ('edges', 'NOUN')
85673 ('together', 'ADV')
85674 (',', '.')
85675 ('right', 'ADJ')
85676 ('side', 'NOUN')
85677 ('up', 'PRT')
85678 ('.', '.')
85679 ('Hardly', 'ADV')
85680 ('an', 'DET')
85681 ('inviting', 'VERB')
85682 ('description', 'NOUN')
85683 ('.', '.')
85684 ('Every', 'DET')
85685 ('time', 'NOUN')
85686 ('you', 'PRON')
85687 ('transplant', 'VERB')
85688 ('a', 'DET')
85689 ('pansy', 'NOUN')
85690 ('you', 'PRON')
85691 ('cause', 'VERB')
85692 ('its', 'DET')
85693 ('flowers', 'NOUN')
85694 ('to', 'PRT')
85695 ('become', 'VERB')
85696 ('smaller', 'ADJ')
8

86375 (',', '.')
86376 ('Spencer', 'NOUN')
86377 ("''", '.')
86378 ('.', '.')
86379 ('It', 'PRON')
86380 ('still', 'ADV')
86381 ('stands', 'VERB')
86382 ('as', 'ADP')
86383 ('a', 'DET')
86384 ('monument', 'NOUN')
86385 ('to', 'ADP')
86386 ('the', 'DET')
86387 ('engineering', 'VERB')
86388 ('skills', 'NOUN')
86389 ('of', 'ADP')
86390 ('the', 'DET')
86391 ('last', 'ADJ')
86392 ('century', 'NOUN')
86393 ('and', 'CONJ')
86394 ('still', 'ADV')
86395 ('serves', 'VERB')
86396 ('in', 'ADP')
86397 ('the', 'DET')
86398 ('gasoline', 'NOUN')
86399 ('age', 'NOUN')
86400 ('to', 'PRT')
86401 ('carry', 'VERB')
86402 ('heavy', 'ADJ')
86403 ('traffic', 'NOUN')
86404 ('on', 'ADP')
86405 ('U.S.', 'NOUN')
86406 ('Route', 'NOUN')
86407 ('250', 'NUM')
86408 ('--', '.')
86409 ('the', 'DET')
86410 ('old', 'ADJ')
86411 ('Beverly', 'NOUN')
86412 ('and', 'CONJ')
86413 ('Fairmont', 'NOUN')
86414 ('Turnpike', 'NOUN')
86415 ('.', '.')
86416 ('Be', 'VERB')
86417 ('sure', 'ADJ')
86418 ('that', 'ADP')
86419 ('this', 'D

87064 ('These', 'DET')
87065 ('were', 'VERB')
87066 ('increased', 'VERB')
87067 ('both', 'DET')
87068 ('in', 'ADP')
87069 ('number', 'NOUN')
87070 ('and', 'CONJ')
87071 ('in', 'ADP')
87072 ('size', 'NOUN')
87073 (',', '.')
87074 ('contained', 'VERB')
87075 ('prominent', 'ADJ')
87076 ('nucleoli', 'NOUN')
87077 (',', '.')
87078 ('and', 'CONJ')
87079 ('were', 'VERB')
87080 ('distributed', 'VERB')
87081 ('throughout', 'ADP')
87082 ('the', 'DET')
87083 ('fiber', 'NOUN')
87084 ('(', '.')
87085 ('Figs.', 'NOUN')
87086 ('2', 'NUM')
87087 ('-', 'ADP')
87088 ('5', 'NUM')
87089 (')', '.')
87090 ('.', '.')
87091 ('He', 'PRON')
87092 ('never', 'ADV')
87093 ('knew', 'VERB')
87094 ('how', 'ADV')
87095 ('he', 'PRON')
87096 ('got', 'VERB')
87097 ('there', 'ADV')
87098 ('.', '.')
87099 ("He'll", 'PRT')
87100 ('get', 'VERB')
87101 ('a', 'DET')
87102 ('fair', 'ADJ')
87103 ('trial', 'NOUN')
87104 ('.', '.')
87105 ('During', 'ADP')
87106 ('his', 'DET')
87107 ('college', 'NOUN')
87108 ('career', 'NOUN')
8710

KeyboardInterrupt: 

In [200]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    
    bigram_results = defaultdict(int)
    
    for seq in sequences:
        for v1, v2 in zip(seq[:-1], seq[1:]):
            bigram_results[(v1,v2)] += 1
    
    return bigram_results
    
    # TODO: Finish this function!
    raise NotImplementedError

# TODO: call bigram_counts with a list of tag sequences from the training set
tag_bigrams = bigram_counts(data.Y)# TODO: YOUR CODE HERE)

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

### IMPLEMENTATION: Sequence Starting Counts
Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

In [199]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    
    starting_count_list = []
    
    for seq in sequences:
        starting_count_list.append(seq[0])
        
    starting_conts_dict = Counter(starting_count_list)
    
    return starting_conts_dict

    # TODO: Finish this function!
    raise NotImplementedError

# TODO: Calculate the count of each tag starting a sequence
tag_starts = starting_counts(data.Y)# TODO: YOUR CODE HERE)

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

### IMPLEMENTATION: Sequence Ending Counts
Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

In [198]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    # TODO: Finish this function!
    
    
    ending_count_list = []
    
    for seq in sequences:
        ending_count_list.append(seq[-1])
        
    ending_conts_dict = Counter(ending_count_list)
    
    return ending_conts_dict
    
    raise NotImplementedError

# TODO: Calculate the count of each tag ending a sequence
tag_ends = ending_counts(data.Y)# TODO: YOUR CODE HERE)

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

### IMPLEMENTATION: Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [66]:
for tag in emission_counts:
    print(emission_counts[tag])

{'Whenever': 12, 'indeed': 92, 'almost': 302, 'Yes': 69, 'About': 34, 'back': 575, 'therefore': 125, 'not': 3562, 'deeply': 29, 'enough': 189, 'so': 1084, 'first': 230, 'then': 798, 'most': 542, 'Meanwhile': 17, 'again': 416, 'more': 968, 'away': 360, 'when': 1410, 'also': 793, 'usually': 151, 'sometimes': 132, 'where': 677, 'preferably': 8, 'because': 116, 'about': 422, 'especially': 116, 'manifestly': 5, 'now': 823, 'why': 194, 'heavily': 47, 'just': 584, 'once': 281, 'only': 1032, 'honestly': 11, 'steadily': 16, 'unblinkingly': 2, 'Just': 91, 'else': 130, 'very': 543, 'much': 350, 'right': 156, 'voraciously': 1, 'well': 551, 'Because': 21, 'doubtless': 9, 'farther': 26, 'negatively': 2, 'later': 248, 'currently': 26, 'thus': 126, 'inevitably': 27, 'far': 298, 'No': 95, 'enterprisingly': 1, 'Even': 150, 'how': 482, 'recently': 88, 'Never': 27, 'Then': 274, 'coldly': 6, 'directly': 109, 'When': 460, 'no': 163, 'longer': 105, 'as': 836, 'high': 27, 'really': 211, 'surely': 33, 'often':

In [67]:
?DiscreteDistribution

In [82]:
print(tag_bigrams.items())

dict_items([(('ADV', 'NOUN'), 1478), (('NOUN', '.'), 62639), (('.', 'ADV'), 5124), (('ADV', '.'), 7577), (('.', 'VERB'), 9041), (('VERB', 'ADP'), 24927), (('ADP', 'ADJ'), 9533), (('ADJ', 'NOUN'), 43664), (('NOUN', 'CONJ'), 13185), (('CONJ', 'VERB'), 6012), (('VERB', 'ADJ'), 8423), (('.', 'DET'), 8008), (('DET', 'VERB'), 7062), (('ADJ', 'PRT'), 1301), (('PRT', 'ADP'), 2189), (('ADP', 'NUM'), 3467), (('NUM', 'NOUN'), 4524), (('.', 'PRON'), 5448), (('PRON', 'VERB'), 27860), (('VERB', 'PRT'), 9556), (('PRT', 'VERB'), 14886), (('VERB', 'NOUN'), 14230), (('NOUN', 'NUM'), 1783), (('NUM', '.'), 3210), (('.', 'NUM'), 1412), (('.', '.'), 12588), (('ADP', 'ADV'), 1805), (('ADV', 'NUM'), 597), (('DET', 'NOUN'), 68785), (('CONJ', 'DET'), 4636), (('NOUN', 'VERB'), 34972), (('ADP', 'NOUN'), 29965), (('ADP', 'DET'), 52841), (('NOUN', 'ADP'), 53884), (('CONJ', 'NOUN'), 7502), (('.', 'NOUN'), 9782), (('VERB', '.'), 11699), (('VERB', 'VERB'), 26957), (('.', 'ADP'), 7595), (('ADV', 'DET'), 3309), (('DET',

In [202]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

# TODO: create states with emission probability distributions P(word | tag) and add to the model
# (Hint: you may need to loop & create/add new states)

all_states = defaultdict()



for tag in emission_counts:
    all_states[tag] = State(DiscreteDistribution(emission_counts[tag]),name=tag)
    basic_model.add_states(all_states[tag])

# TODO: add edges between states for the observed transition frequencies P(tag_i | tag_i-1)
# (Hint: you may need to loop & add transitions

for tag in all_states:
    start_prob = tag_starts[tag] * 1.0 / len(tag_starts) # probability of the starting edge
    basic_model.add_transition(basic_model.start, all_states[tag], start_prob)

for tag in all_states:
    end_prob = tag_ends[tag] * 1.0 / len(data.training_set.Y) # probability of the ending edge
    basic_model.add_transition(all_states[tag], basic_model.end, end_prob)

#     print(tag) # debugging only

for (tag_A,tag_B), bigram_freq_counts in tag_bigrams.items():
    tag_transition_prob = bigram_freq_counts*1.0/(tag_unigrams[tag_A])
    basic_model.add_transition(all_states[tag_A],all_states[tag_B], tag_transition_prob)
    
# NOTE: YOU SHOULD NOT NEED TO MODIFY ANYTHING BELOW THIS LINE
# finalize the model
basic_model.bake()

assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">Your HMM network topology looks good!</div>')

In [203]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_training_acc > 0.955, "Uh oh. Your HMM accuracy on the training set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct! Congratulations, you\'ve finished the project.</div>')

training accuracy basic hmm model: 96.97%
testing accuracy basic hmm model: 95.59%


AssertionError: Uh oh. Your HMM accuracy on the training set doesn't look right.

### Example Decoding Sequences with the HMM Tagger

In [90]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '.', 


## Finishing the project
---

<div class="alert alert-block alert-info">
**Note:** **SAVE YOUR NOTEBOOK**, then run the next cell to generate an HTML copy. You will zip & submit both this file and the HTML copy for review.
</div>

In [ ]:
!!jupyter nbconvert *.ipynb

## Step 4: [Optional] Improving model performance
---
There are additional enhancements that can be incorporated into your tagger that improve performance on larger tagsets where the data sparsity problem is more significant. The data sparsity problem arises because the same amount of data split over more tags means there will be fewer samples in each tag, and there will be more missing data  tags that have zero occurrences in the data. The techniques in this section are optional.

- [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (pseudocounts)
    Laplace smoothing is a technique where you add a small, non-zero value to all observed counts to offset for unobserved values.

- Backoff Smoothing
    Another smoothing technique is to interpolate between n-grams for missing data. This method is more effective than Laplace smoothing at combatting the data sparsity problem. Refer to chapters 4, 9, and 10 of the [Speech & Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book for more information.

- Extending to Trigrams
    HMM taggers have achieved better than 96% accuracy on this dataset with the full Penn treebank tagset using an architecture described in [this](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf) paper. Altering your HMM to achieve the same performance would require implementing deleted interpolation (described in the paper), incorporating trigram probabilities in your frequency tables, and re-implementing the Viterbi algorithm to consider three consecutive states instead of two.

### Obtain the Brown Corpus with a Larger Tagset
Run the code below to download a copy of the brown corpus with the full NLTK tagset. You will need to research the available tagset information in the NLTK docs and determine the best way to extract the subset of NLTK tags you want to explore. If you write the following the format specified in Step 1, then you can reload the data using all of the code above for comparison.

Refer to [Chapter 5](http://www.nltk.org/book/ch05.html) of the NLTK book for more information on the available tagsets.

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]